In [14]:
import pandas as pd 
import ast
import pymysql
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import tqdm
import os


In [79]:
def scrap_medicament(url, code_barre=None, temps_sleep=3):
        options = Options()
        options.add_argument("--headless")  # mode sans interface
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        # Initialisation du navigateur
        driver = webdriver.Chrome(options=options)
        driver.get(url)
        time.sleep(temps_sleep)

        html = driver.page_source
        driver.quit()

        soup = BeautifulSoup(html, "html.parser")
        data = {}
        h3_tag = soup.find("h3")
        data["Code_EAN_13"] = code_barre if code_barre else None
        data["Nom"] = h3_tag.get_text(strip=True) if h3_tag else None
        img_tag = soup.select_one("div.only-mobile img")
        data["Image URL"] = img_tag["src"] if img_tag else None
        rows = soup.select("table.table-details tr")
        for row in rows:
            field = row.find("td", class_="field")
            value = row.find("td", class_="value")
            if field and value:
                key = field.get_text(strip=True)
                val = value.get_text(strip=True)
                data[key] = val

        def parse_prix(prix_str):
            """Convertit une chaîne de type '36.20 dhs' en float 36.20"""
            if prix_str:
                return float(prix_str.replace("dhs", "").strip())
            return None

        
        data["url"] = url
        return data

In [15]:
host = "srv1905.hstgr.io"
user = "u791384722_pharma"
password = "@sCM^n4L"
database = "u791384722_pharma"

request_insert ="""INSERT INTO Medicament (
    Code_EAN_13,
    Nom,
    Image_URL,
    Présentation,
    Dosage,
    Distributeur_ou_fabriquant,
    Composition,
    Classe_thérapeutique,
    Statut,
    Code_ATC,
    PPV,
    Prix_hospitalier,
    Tableau,
    Indications,
    Min_Stock,
    Stock_Actuel, 
    url_medicament
) VALUES (
    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
)"""

In [45]:
path_folder = '/Users/ahmedzaiou/Documents/ProjetsApps/MiseajourMedicament/data_from_eservice'
list_of_pd = []
for path in os.listdir(path_folder):
    if path.endswith('.json'): 
        df = pd.read_json(os.path.join(path_folder, path)) 
        if not df.empty :
            list_of_pd.append(df) 
all_data = pd.concat(list_of_pd, ignore_index=True)
all_data_old = all_data.copy()

In [46]:
deja_dans_la_base  = pd.read_csv("/Users/ahmedzaiou/Documents/ProjetsApps/TrackPharmic/Datascraping/Medicament (3).csv")['Code_EAN_13'].values

In [73]:
all_data['code_ean_13'] =  all_data['code_ean_13'].astype(str)

In [74]:
all_data = all_data[all_data['code_ean_13'].isin(deja_dans_la_base) == False]

In [75]:
all_data.shape, all_data_old.shape

((1963, 9), (6063, 9))

In [76]:
data_filtred = all_data.copy()

In [95]:
data_filtred

,nom,code_ean_13,remboursable,prix_public,prix_remb_officine,prix_hospitalier,substance_active,classe_therapeutique,generique
2,MIRZAM 10MG : 1 BOITE 30 GELULE GELULE à 10 MG,6118000400042,True,106 DH,66.2 DH,66.2 DH,RAMIPRIL,INHIBITEUR DE L ENZYME DE CONVERSION,True
3,"MIRZAM 1,25MG : 1 BOITE 30 GELULE GELULE à 1.2...",6118000400011,True,40 DH,25 DH,25 DH,RAMIPRIL,INHIBITEUR DE L ENZYME DE CONVERSION,True
6,APIDRA : 1 FLACON 10 ML SOLUTION INJECTABLE UR...,6118001081462,True,235 DH,147.5 DH,147.5 DH,INSULINE HUMAINE,INSULINES ET ANALOGUES,NaN
23,AVLOCARDYL LP : 1 BOITE 30 GELULE LP GELULE LP...,6118001180530,True,88.2 DH,55.1 DH,55.1 DH,PROPRANOLOL,BETABLOQUANT,NaN
30,XYLOCAINE 2% ADRENALINE : 1 BOITE 1 FLACON SOL...,6118001020713,True,16.1 DH,10 DH,10 DH,ADRENALINE - LIDOCAINE,ANESTHESIQUE LOCAL,NaN
...,...,...,...,...,...,...,...,...,...
6032,NOVOCLIN : 1 BOITE 14 SACHET POUDRE POUR SUSPE...,6118000140351,True,127.5 DH,84.3 DH,84.3 DH,AMOXICILLINE - ACIDE CLAVULANIQUE,PENICILLINE,True
6033,NOVOCLIN 100MG-12.5MG : 1 FLACON 60 ML POUDRE ...,6118000141358,True,58.9 DH,36.8 DH,36.8 DH,AMOXICILLINE - ACIDE CLAVULANIQUE,PENICILLINE,True
6038,NOVOCLIN 1G-125MG : 1 BOITE 12 SACHET POUDRE P...,6118000141419,True,92.5 DH,57.8 DH,57.8 DH,AMOXICILLINE - ACIDE CLAVULANIQUE,PENICILLINE,True
6058,INTETRIX : 1 BOITE 20 GELULE GELULE à 50 - 200...,6118000010852,True,31.7 DH,19.8 DH,19.8 DH,TILIQUINOL - TILBROQUINOL,ANTIPARASITAIRE,NaN


In [117]:
conn = pymysql.connect(
                    host=host, user=user, password=password, database=database
                )   

In [ ]:
conn = pymysql.connect(
                    host=host, user=user, password=password, database=database
                )   
for index, values in data_filtred.iterrows():
    
    values_inser = (
            values['code_ean_13'],
            values['nom'],
            "",
            "",
            "",
            "",
            "",
            values['classe_therapeutique'],
            values['substance_active'],
            "",
            values['prix_public'],
            values['prix_hospitalier'],
            "",
            "",
            0,
            0,
            "",
        ) 
    values_update = (
            
            values['nom'],
            "",
            "",
            "",
            "",
            "",
            values['classe_therapeutique'],
            values['substance_active'],
            "",
            values['prix_public'],
            values['prix_hospitalier'],
            values['generique'],
            "",
            0,
            0,
            "",
            values['code_ean_13'],
        ) 
    cursor = conn.cursor(pymysql.cursors.DictCursor)
    
    # 1. Vérifier si le code existe
    cursor.execute("SELECT 1 FROM Medicament WHERE Code_EAN_13 = %s", (values['code_ean_13'],))
    exists = cursor.fetchone()

    # 2. Si pas trouvé, insérer
    if not exists: 
        cursor.execute(request_insert, values_inser)
        conn.commit()
    else:
        pass
        #print(values['code_ean_13'], "exist")
        #cursor.execute(
        #    request_update,
        #    values_update,
        #)
        #conn.commit()
    

    

In [119]:
all_medicament = pd.read_csv("/Users/ahmedzaiou/Documents/ProjetsApps/TrackPharmic/Datascraping/Medicament (4).csv")

In [122]:
import json

In [124]:
para_path = '/Users/ahmedzaiou/Documents/ProjetsApps/TrackPharmic/Datascraping/scrap_no_medicament/results_jsons'
list_of_para = []
for path in os.listdir(para_path):
    if path.endswith('.json'): 
        # read json     
        with open(os.path.join(para_path, path)) as f:
            data = json.load(f)
            list_of_para.append(data)

all_para = pd.DataFrame(list_of_para)


In [128]:
all_para = all_para.dropna(subset=['prix'])

In [129]:
all_para.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8423 entries, 0 to 8426
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   titre             8423 non-null   object
 1   prix              8423 non-null   object
 2   devise            0 non-null      object
 3   code_bar          8423 non-null   object
 4   categories        8423 non-null   object
 5   tags              8423 non-null   object
 6   images            8400 non-null   object
 7   descriptions      8423 non-null   object
 8   caracteristiques  8423 non-null   object
 9   url               8423 non-null   object
dtypes: object(10)
memory usage: 723.9+ KB


In [131]:
conn = pymysql.connect(
                    host=host, user=user, password=password, database=database
                )   
for index, values in all_para.iterrows():
    
    values_inser = (
            values['code_bar'],
            values['titre'],
            values['images'] if values['images'] else "",
            values['categories'],
            "",
            "",
            values['tags'],
            "",
            "",
            "",
            values['prix'],
            values['prix'],
            "",
            values['descriptions'],
            0,
            0,
            values['url'],
        ) 
     
    cursor = conn.cursor(pymysql.cursors.DictCursor)
    
    # 1. Vérifier si le code existe
    cursor.execute("SELECT 1 FROM Medicament WHERE Code_EAN_13 = %s", (values['code_bar'],))
    exists = cursor.fetchone()

    # 2. Si pas trouvé, insérer
    if not exists: 
        cursor.execute(request_insert, values_inser)
        conn.commit()
    else:
        print(values['code_bar'])
        #cursor.execute(
        #    request_update,
        #    values_update,
        #)
        #conn.commit()
    

    

3700221322994
6111261530022
6111259970106
8052877170494
6111261530374
